In [55]:
# Import libraries
import os
from PyPDF2 import PdfReader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from ollama import Client


In [56]:
# Use PyPDF2 to extract text from the PDFs
# Function to extract text from a PDF file
# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ''
    for page in reader.pages:
        text += page.extract_text()
    return text
    
# Directory where PDF files are located
pdf_dir = '/Users/danieldan/Lavori/temp'
pdf_texts = []
pdf_filenames = []

# Extract text from each PDF and store the filenames
for pdf_file in os.listdir(pdf_dir):
    if pdf_file.endswith(".pdf"):
        pdf_path = os.path.join(pdf_dir, pdf_file)
        pdf_text = extract_text_from_pdf(pdf_path)
        pdf_texts.append(pdf_text)
        pdf_filenames.append(pdf_file)

print("Text extraction complete. Found documents:", pdf_filenames)

Text extraction complete. Found documents: ['8.a. Study_Regulations_MBA_20240409 for University Senate July 2024.pdf', '5.c. Study Regulations for the Bachelor in Applied Data Science (1).pdf', '5.a. Study Regulations Bachelor of Business Administration in Tourism, Hotel Management, and Operations.pdf', '9.a. Study Regulations PhD_20230703.pdf', '5.d. Study Regulations for the Bachelor of Science in International Management PE.pdf', '7.a. Study Regulations MSc_20240409 + Data Science for University Senate July 2024.pdf', '5.b. Study Regulations for the Bachelor of Business Administration in THM.pdf', '6.b. Study Regulations for the Bachelor of Science in HR management and Business Psychology.pdf', '6.a. Study Regulations for the Bachelor of Arts in International Relation and Sustainability.pdf', '5.e. Study Regulations for the Bachelor of Science in International Management.pdf', 'US 20240409 Agenda.pdf', 'US Report by the President 20240701.pdf']


In [10]:
# Use PyPDF2 to extract text from the PDFs
# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ''
    for page in reader.pages:
        text += page.extract_text()
    return text

# Directory where PDF files are located
pdf_dir = '/Users/danieldan/Lavori/temp'
pdf_texts = []
pdf_filenames = []

# Extract text from each PDF and store the filenames
for pdf_file in os.listdir(pdf_dir):
    if pdf_file.endswith(".pdf"):
        pdf_path = os.path.join(pdf_dir, pdf_file)
        pdf_text = extract_text_from_pdf(pdf_path)
        pdf_texts.append(pdf_text)
        pdf_filenames.append(pdf_file)

print("Text extraction complete. Found documents:", pdf_filenames)

Text extraction complete. Found documents: ['8.a. Study_Regulations_MBA_20240409 for University Senate July 2024.pdf', '5.c. Study Regulations for the Bachelor in Applied Data Science (1).pdf', '5.a. Study Regulations Bachelor of Business Administration in Tourism, Hotel Management, and Operations.pdf', '9.a. Study Regulations PhD_20230703.pdf', '5.d. Study Regulations for the Bachelor of Science in International Management PE.pdf', '7.a. Study Regulations MSc_20240409 + Data Science for University Senate July 2024.pdf', '5.b. Study Regulations for the Bachelor of Business Administration in THM.pdf', '6.b. Study Regulations for the Bachelor of Science in HR management and Business Psychology.pdf', '6.a. Study Regulations for the Bachelor of Arts in International Relation and Sustainability.pdf', '5.e. Study Regulations for the Bachelor of Science in International Management.pdf', 'US 20240409 Agenda.pdf', 'US Report by the President 20240701.pdf']


In [57]:
# Generate embeddings using SentenceTransformers

# Initialize the SentenceTransformer model
embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# Generate embeddings for the extracted text
#pdf_embeddings = embedding_model.embed_documents(pdf_texts)

In [58]:
# Store embeddings in ChromaDB using LangChain
# Use LangChain’s Chroma class to interact with ChromaDB
# Initialize ChromaDB client and collection using LangChain's Chroma
vectorstore = Chroma(
    embedding_function=embedding_model,
    collection_name="pdf_documents",
    persist_directory="./chromadb"
)

# Add the documents to ChromaDB
for i, text in enumerate(pdf_texts):
    vectorstore.add_texts(texts=[text], ids=[pdf_filenames[i]])


In [59]:
# Use LangChain’s RetrievalQA with Ollama
# We will use LangChain’s RetrievalQA chain and Ollama’s Client for querying the LLaMA3.1 model.
# Initialize the Ollama client

# Initialize Ollama client
ollama_client = Client()

# Function to query LLaMA3 via Ollama's Client
#def get_llama3_response(prompt):
#    response = ollama_client.generate(model="llama3.1", prompt=prompt)
#    return response['text']

# Example user question
user_question = "How is the thesis supervision done?"

# Query ChromaDB using the proper LangChain API
results = vectorstore.similarity_search(query=user_question, k=3)  # k=3 means top 3 results

In [60]:
# Query ChromaDB for relevant documents
# Example user question

# Process and generate a response using Ollama
if results:
    prompt = "You are an assistant. Use the following documents to answer the question:\n"
    for result in results:
        prompt += f"{result.page_content}\n"
    prompt += f"\nAnswer the following question: {user_question}"

    # Get the response from the Ollama model
    response = ollama_client.generate(model="llama3.1", prompt=prompt)
    if 'response' in response:
        print(f"Answer: {response['response']}")
    else:
        print("No 'response' key found in the response.")
else:
    print("No relevant information found.")

# Check if documents are retrieved
if results:
    print("\nRetrieved documents:")
    for i, result in enumerate(results, 1):
        print(f"Document {i} content: {result.page_content[:200]}...")  # Print first 200 characters
else:
    print("No relevant documents found.")

print("\nPDF texts content:")
for i, text in enumerate(pdf_texts, 1):
    print(f"PDF {i} content: {text[:200]}...")  # Print first 200 characters

Answer: According to the provided study regulations, the thesis supervision process involves the following steps:

* The student (or group of students) must find a suitable topic for their master thesis. If they are unable to do so, the Dean will assign them to a lecturer who will then assign a topic.
* Once a topic is chosen, the student and their supervisor must announce it to the Dean in writing, along with a supervision statement from the faculty member or external lecturer serving as the supervisor.
* The student can only change their thesis topic once, and only within the first three months after announcing the topic to the Dean.
* The master thesis must be written under the guidance of a supervisor who holds an academic doctoral degree or equivalent.

Retrieved documents:
Document 1 content: 1  
 Study  Regulations  for the Bachelor  of Science  in International  
Management  
Decided on XX,XX, 2024,  by the University  Senate  
 
Due to the accreditation  as a private  universi